## Data Ingestion

In [1]:
%pwd

'd:\\Imarticus_Learning\\Hackathon\\Interview_Intelligence_Prediction\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Imarticus_Learning\\Hackathon\\Interview_Intelligence_Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from interviewIntelligence.constants import *
from interviewIntelligence.utils.common import read_yaml, create_directries
from interviewIntelligence.logging import logger

In [8]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directries([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [9]:
import zipfile 
import urllib.request as requests

In [10]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        try:
            create_directries([self.config.root_dir])
            
            if not os.path.exists(self.config.local_data_file):
                file_name, header = requests.urlretrieve(
                    self.config.source_URL,
                    self.config.local_data_file
                )
                logger.info(f"{file_name} downloaded with following info {header}.")
            else:
                logger.info("File already exists")
        except Exception as e:
            raise e

    def extract_zip_file(self):
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(self.config.unzip_dir, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
                zip_ref.extractall(unzip_path)    
        except Exception as e:
            raise e

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-11-03 16:35:18,608]: INFO common Created directory at: artifacts
[2023-11-03 16:35:18,609]: INFO common Created directory at: artifacts/data_ingestion
[2023-11-03 16:35:19,809]: INFO 2376431056 artifacts/data_ingestion/data.zip downloaded with following info Connection: close
Content-Length: 320163
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "02ba62163b027c92eecc85676fbcd82647180cd2db8d52d48609c2b691c55387"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8A8A:3F9275:5C5A:8671:6544D3F3
Accept-Ranges: bytes
Date: Fri, 03 Nov 2023 11:05:24 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100032-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1699009524.992500,VS0,VE358
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-